# Compound Triage Dashboard

Interactive dashboard for evaluating compounds using Lipinski's Rule of Five. Works as both a Jupyter notebook and a Voila web app.


In [ ]:
import os
from amprenta_rag.client import RAGClient
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
import numpy as np

# Connect to API
api_url = os.environ.get('API_URL', 'http://host.docker.internal:8000')
client = RAGClient(api_url=api_url)
print(f'Connected to {api_url}')


In [ ]:
# Load compounds
compounds = client.compounds.list()
print(f'Loaded {len(compounds)} compounds')


In [ ]:
def score_mw(mw):
    """Score molecular weight: green (<500), yellow (500-600), red (>600)"""
    if mw is None:
        return None, "gray"
    if mw < 500:
        return "✓", "green"
    elif mw <= 600:
        return "⚠", "orange"
    else:
        return "✗", "red"

def score_logp(logp):
    """Score LogP: green (0-3), yellow (3-5), red (>5)"""
    if logp is None:
        return None, "gray"
    if 0 <= logp <= 3:
        return "✓", "green"
    elif 3 < logp <= 5:
        return "⚠", "orange"
    else:
        return "✗", "red"

def score_hbd(hbd):
    """Score HBD: green (<=5), yellow (6-7), red (>7)"""
    if hbd is None:
        return None, "gray"
    if hbd <= 5:
        return "✓", "green"
    elif hbd <= 7:
        return "⚠", "orange"
    else:
        return "✗", "red"

def score_hba(hba):
    """Score HBA: green (<=10), yellow (11-12), red (>12)"""
    if hba is None:
        return None, "gray"
    if hba <= 10:
        return "✓", "green"
    elif hba <= 12:
        return "⚠", "orange"
    else:
        return "✗", "red"

def score_rotatable_bonds(rot_bonds):
    """Score rotatable bonds: green (<=10), yellow (11-15), red (>15)"""
    if rot_bonds is None:
        return None, "gray"
    if rot_bonds <= 10:
        return "✓", "green"
    elif rot_bonds <= 15:
        return "⚠", "orange"
    else:
        return "✗", "red"

def calculate_lipinski_score(compound):
    """Calculate overall Lipinski score (0-5 rules passed)."""
    score = 0
    if compound.molecular_weight and compound.molecular_weight < 500:
        score += 1
    if compound.logp is not None and 0 <= compound.logp <= 5:
        score += 1
    if compound.hbd_count is not None and compound.hbd_count <= 5:
        score += 1
    if compound.hba_count is not None and compound.hba_count <= 10:
        score += 1
    if compound.rotatable_bonds is not None and compound.rotatable_bonds <= 10:
        score += 1
    return score


In [ ]:
# Create output widget
output_area = widgets.Output()

def render_triage_table():
    """Render compound triage table with traffic-light scoring."""
    with output_area:
        clear_output(wait=True)
        
        if not compounds:
            print("No compounds found.")
            return
        
        # Build triage data
        triage_data = []
        for comp in compounds:
            mw_symbol, mw_color = score_mw(comp.molecular_weight)
            logp_symbol, logp_color = score_logp(comp.logp)
            hbd_symbol, hbd_color = score_hbd(comp.hbd_count)
            hba_symbol, hba_color = score_hba(comp.hba_count)
            rot_symbol, rot_color = score_rotatable_bonds(comp.rotatable_bonds)
            lipinski_score = calculate_lipinski_score(comp)
            
            triage_data.append({
                'Compound ID': comp.compound_id or 'N/A',
                'MW': comp.molecular_weight if comp.molecular_weight else 'N/A',
                'MW Score': mw_symbol or 'N/A',
                'LogP': comp.logp if comp.logp is not None else 'N/A',
                'LogP Score': logp_symbol or 'N/A',
                'HBD': comp.hbd_count if comp.hbd_count is not None else 'N/A',
                'HBD Score': hbd_symbol or 'N/A',
                'HBA': comp.hba_count if comp.hba_count is not None else 'N/A',
                'HBA Score': hba_symbol or 'N/A',
                'Rot Bonds': comp.rotatable_bonds if comp.rotatable_bonds is not None else 'N/A',
                'Rot Score': rot_symbol or 'N/A',
                'Lipinski Score': lipinski_score
            })
        
        # Create DataFrame
        df = pd.DataFrame(triage_data)
        
        # Sort by Lipinski score (descending)
        df = df.sort_values('Lipinski Score', ascending=False)
        
        # Create HTML table with colored cells
        html_rows = []
        html_rows.append('<table border="1" style="border-collapse: collapse; width: 100%;">')
        html_rows.append('<thead><tr>')
        for col in df.columns:
            html_rows.append(f'<th style="padding: 8px; background-color: #f0f0f0;">{col}</th>')
        html_rows.append('</tr></thead><tbody>')
        
        for idx, row in df.iterrows():
            html_rows.append('<tr>')
            html_rows.append(f'<td style="padding: 8px;">{row["Compound ID"]}</td>')
            
            # MW column
            mw_val = row['MW']
            mw_sym = row['MW Score']
            mw_color = score_mw(mw_val)[1] if mw_val != 'N/A' else 'gray'
            html_rows.append(f'<td style="padding: 8px;">{mw_val}</td>')
            html_rows.append(f'<td style="padding: 8px; background-color: {mw_color}; text-align: center;">{mw_sym}</td>')
            
            # LogP column
            logp_val = row['LogP']
            logp_sym = row['LogP Score']
            logp_color = score_logp(logp_val)[1] if logp_val != 'N/A' else 'gray'
            html_rows.append(f'<td style="padding: 8px;">{logp_val}</td>')
            html_rows.append(f'<td style="padding: 8px; background-color: {logp_color}; text-align: center;">{logp_sym}</td>')
            
            # HBD column
            hbd_val = row['HBD']
            hbd_sym = row['HBD Score']
            hbd_color = score_hbd(hbd_val)[1] if hbd_val != 'N/A' else 'gray'
            html_rows.append(f'<td style="padding: 8px;">{hbd_val}</td>')
            html_rows.append(f'<td style="padding: 8px; background-color: {hbd_color}; text-align: center;">{hbd_sym}</td>')
            
            # HBA column
            hba_val = row['HBA']
            hba_sym = row['HBA Score']
            hba_color = score_hba(hba_val)[1] if hba_val != 'N/A' else 'gray'
            html_rows.append(f'<td style="padding: 8px;">{hba_val}</td>')
            html_rows.append(f'<td style="padding: 8px; background-color: {hba_color}; text-align: center;">{hba_sym}</td>')
            
            # Rotatable bonds column
            rot_val = row['Rot Bonds']
            rot_sym = row['Rot Score']
            rot_color = score_rotatable_bonds(rot_val)[1] if rot_val != 'N/A' else 'gray'
            html_rows.append(f'<td style="padding: 8px;">{rot_val}</td>')
            html_rows.append(f'<td style="padding: 8px; background-color: {rot_color}; text-align: center;">{rot_sym}</td>')
            
            # Lipinski score column
            score = row['Lipinski Score']
            score_color = 'green' if score >= 4 else 'orange' if score >= 3 else 'red'
            html_rows.append(f'<td style="padding: 8px; background-color: {score_color}; text-align: center; font-weight: bold;">{score}/5</td>')
            
            html_rows.append('</tr>')
        
        html_rows.append('</tbody></table>')
        
        html_table = ''.join(html_rows)
        display(HTML(html_table))
        
        # Summary statistics
        total = len(df)
        high_score = len(df[df['Lipinski Score'] >= 4])
        medium_score = len(df[(df['Lipinski Score'] >= 3) & (df['Lipinski Score'] < 4)])
        low_score = len(df[df['Lipinski Score'] < 3])
        
        print(f"\n## Summary")
        print(f"**Total Compounds:** {total}")
        print(f"**High Score (≥4/5):** {high_score} ({high_score/total*100:.1f}%)")
        print(f"**Medium Score (3/5):** {medium_score} ({medium_score/total*100:.1f}%)")
        print(f"**Low Score (<3/5):** {low_score} ({low_score/total*100:.1f}%)")

# Display table
display(output_area)
render_triage_table()
